In [ ]:
import tensorflow as tf
tf.reset_default_graph()
import numpy as np

print('x')

# Input configuration.
TEXT_SIZE = 16
KEY_SIZE  = 16

# Training parameters.
learning_rate   = 0.0008
batch_size      = 4096
sample_size     = 4096*5
epochs          = 850000
steps_per_epoch = int(sample_size/batch_size)

ITERS_PER_ACTOR = 1
EVE_MULTIPLIER = 2  # Train Eve 2x for every step of Alice/Bob

# Function to return a list of tensor's to be used in training based on the scope of model
def get_collection(collection):
  return tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=collection) 

# Create two placeholders to hold message and key
input_message = tf.placeholder(tf.float32, shape=(batch_size, TEXT_SIZE), name='input_message')
input_key     = tf.placeholder(tf.float32, shape=(batch_size, KEY_SIZE), name='input_key')


# Function to create individual models for Alice, Bob and Eve.
def model(collection, message, key=None):
  
  if key is not None:
    # if there is a key then it's either Alice or Bob model trying to encrypt and decrypt resp
    # message and key is concatenated into single tensor and fed as an input
    combined_input = tf.concat(axis=1, values=[message, key])
  else:
    # if no key is present then it's Eve model trying to eavesdrop hence pass the message as input tensor 
    combined_input = message
  
  # collection arg is to denote the scope of model which is used to aggregate the 
  # tensor to make sure all tensor which needs to be trained are inside one scope
  with tf.variable_scope(collection):
    
    
    ## Fully connected layer of 16+16 = 32 neurons
    fc = tf.layers.dense(combined_input, TEXT_SIZE + KEY_SIZE, activation=tf.nn.relu)
    # expand and create channel to be valid for convolution
    # i.e convert FC layer a vector of 32 to matrix of (32,1) 
    fc = tf.expand_dims(fc, 2)
    
    
    ## Convolution Layers - Sigmoid activation function
    
    # input: (32,1) -> output:(32,2) because filter is 2 which creates 2 channels
    conv1 = tf.layers.conv1d( fc,    filters=2, kernel_size=4, strides= 1, padding='SAME',  activation=tf.nn.sigmoid)
    
    # input: (32,2) -> output:(16,4) because stride is 2 hence result is halved ( i.e 32/2 )
    # filter is 4 which creates 4 channels
    conv2 = tf.layers.conv1d( conv1, filters=4, kernel_size=2, strides= 2, padding='VALID', activation=tf.nn.sigmoid)
    
    # input: (16,4) -> output:(16,4) because filter is 4 which creates 4 channels
    conv3 = tf.layers.conv1d( conv2, filters=4, kernel_size=1, strides= 1, padding='SAME',  activation=tf.nn.sigmoid)
    
    
    ## Convolution Layers - Tanh activation function
    
    # input: (16,4) -> output:(16,1) because filter is 1 which creates 1 channel
    conv4 = tf.layers.conv1d( conv3, filters=1, kernel_size=1, strides=1, padding='SAME', activation=tf.nn.tanh)
    
    # Opposite of expand_dims function, here (16,1) tensor is converted to tensor of (16)
    conv4 = tf.squeeze(conv4, 2)
    
    
    return conv4

def symm_crypto_create(input_message, input_key):
  
  # Alice takes in message and shared secret key to encrypt and create a ciphertext
  Alice_out_ciphertext = model( 'Alice', input_message, input_key)

  # Bob takes in ciphertext and shared secret key to decrypt and recreate the original message
  Bob_out_message      = model( 'Bob',   Alice_out_ciphertext, input_key)

  # Eve only takes in ciphertext and tries to decrypt it without using key
  Eve_out_message      = model( 'Eve',   Alice_out_ciphertext)
  
  return Bob_out_message, Eve_out_message

def random_bools(size, n):
  
  # create a batch size of 'size' with 'n' number of bits per sample
  temp =  np.random.random_integers(0, high=1, size=[size, n])
  
  # Convert 0 -> -1 and 1 -> 1 
  temp = temp*2 - 1
  
  return temp.astype(np.float32)
  
def get_dataset(sample_size, TEXT_SIZE, KEY_SIZE):

  m = random_bools(sample_size, TEXT_SIZE)
  k = random_bools(sample_size, KEY_SIZE)
  
  return m, k

def eve_loss(input_message, eve_out_message, batch_size):

  # Mapping -1 to 0 and 1 to 1
  eve_diff = (eve_out_message + 1.0)/2.0 - (input_message + 1.0)/2.0
  
  # Eve's average L1 distance Loss of the given batch
  loss = (1/batch_size)*tf.reduce_sum(tf.abs(eve_diff))
  
  return loss

def alice_bob_loss(input_message, bob_out_message, Eve_loss, batch_size):

  ## Alice-Bob Loss part-1
  
  # Mapping -1 to 0 and 1 to 1
  ab_diff = (bob_out_message + 1.0)/2.0 - (input_message + 1.0)/2.0
  
  # Alice and Bob's average L1 distance Loss of the given batch
  bob_reconstruction_loss = (1/batch_size)*tf.reduce_sum(tf.abs(ab_diff))

  ## Alice-Bob Loss part-2
  # To make sure Eve at least has 50% of bits wrong so that output simulates random probability of binary output
  # ((N/2 - EveLoss)^2)/((N/2)^2)
  eve_evedropping_loss = tf.reduce_sum( tf.square(float(TEXT_SIZE) / 2.0 - Eve_loss) / (TEXT_SIZE / 2)**2)

  # bobs final reconstruction loss
  loss = bob_reconstruction_loss + eve_evedropping_loss
  
  return loss


sess = tf.Session()

# Create symmetric cryptosystem in form of adversarial networks
Bob_out, Eve_out = symm_crypto_create(input_message, input_key)

# Loss
eve_loss       = eve_loss(input_message, Eve_out, batch_size)
alice_bob_loss = alice_bob_loss(input_message, Bob_out, eve_loss, batch_size)

# Collect each model's tensors
alice_vars = get_collection('Alice')
bob_vars   = get_collection('Bob')
eve_vars   = get_collection('Eve')

# optimizers
Eve_opt  = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.9, epsilon=1e-08).minimize(eve_loss, var_list=[eve_vars])
bob_opt  = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.9, epsilon=1e-08).minimize(alice_bob_loss, var_list=[alice_vars + bob_vars]) 

# initialize variables
init = tf.global_variables_initializer()
sess.run(init)


# DATASET 
messages, keys = get_dataset(sample_size, TEXT_SIZE, KEY_SIZE)

for i in range(epochs):

  for j in range(steps_per_epoch):
    
    # select the batch from the samples
    batch_messages = messages[j*batch_size: (j+1)*batch_size]
    batch_keys     = keys[j*batch_size: (j+1)*batch_size]

    # Train Alice and Bob's models
    for _ in range(ITERS_PER_ACTOR):
      temp = sess.run([bob_opt, alice_bob_loss],feed_dict={input_message:batch_messages , input_key:batch_keys })
      Alice_bob_loss = temp[1]

    # Train Eve's model
    for _ in range(ITERS_PER_ACTOR*EVE_MULTIPLIER):
      temp = sess.run([Eve_opt, eve_loss], feed_dict={input_message:batch_messages , input_key:batch_keys })
      Eve_loss = temp[1]

  # output Alice-Bob loss and Eve's loss after every 100 iterations  
  if i%100 == 0:
    print(i,'  Alice_bob_loss: ', Alice_bob_loss,'    Eve_loss:', Eve_loss)

sess.close()

W0106 18:52:13.704952 21936 deprecation.py:323] From <ipython-input-1-c9d51d2b37f6>:47: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.
W0106 18:52:13.728312 21936 deprecation.py:506] From C:\Users\shrey\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


x


W0106 18:52:14.347517 21936 deprecation.py:323] From <ipython-input-1-c9d51d2b37f6>:56: conv1d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv1D` instead.
C:\Users\shrey\Anaconda3\lib\site-packages\ipykernel_launcher.py:93: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead


0   Alice_bob_loss:  7.9897003     Eve_loss: 8.0262785
100   Alice_bob_loss:  5.441805     Eve_loss: 3.2429485
200   Alice_bob_loss:  0.70159644     Eve_loss: 3.9050083
300   Alice_bob_loss:  0.27835345     Eve_loss: 5.203371
400   Alice_bob_loss:  0.1624307     Eve_loss: 5.7575045
500   Alice_bob_loss:  0.10843047     Eve_loss: 6.080491
600   Alice_bob_loss:  0.06253946     Eve_loss: 6.6213164
700   Alice_bob_loss:  0.045173075     Eve_loss: 6.791985
800   Alice_bob_loss:  0.032793947     Eve_loss: 6.9683104
900   Alice_bob_loss:  0.02484926     Eve_loss: 7.090041
1000   Alice_bob_loss:  0.01951921     Eve_loss: 7.180002
1100   Alice_bob_loss:  0.016116396     Eve_loss: 7.2320285
1200   Alice_bob_loss:  0.012232971     Eve_loss: 7.331219
1300   Alice_bob_loss:  0.009621222     Eve_loss: 7.4032917
1400   Alice_bob_loss:  0.007843776     Eve_loss: 7.4546127
1500   Alice_bob_loss:  0.006453174     Eve_loss: 7.4964256
1600   Alice_bob_loss:  0.005459455     Eve_loss: 7.5263596
1700   Alic

13700   Alice_bob_loss:  0.00953649     Eve_loss: 7.2189255
13800   Alice_bob_loss:  0.0072840713     Eve_loss: 7.3177443
13900   Alice_bob_loss:  0.005938668     Eve_loss: 7.3855104
14000   Alice_bob_loss:  0.004972232     Eve_loss: 7.437976
14100   Alice_bob_loss:  0.004357137     Eve_loss: 7.47462
14200   Alice_bob_loss:  0.0038407051     Eve_loss: 7.505695
14300   Alice_bob_loss:  0.003253446     Eve_loss: 7.547762
14400   Alice_bob_loss:  0.002417767     Eve_loss: 7.6109977
14500   Alice_bob_loss:  0.009217844     Eve_loss: 7.232827
14600   Alice_bob_loss:  0.007261046     Eve_loss: 7.3194747
14700   Alice_bob_loss:  0.006014917     Eve_loss: 7.3815546
14800   Alice_bob_loss:  0.004989764     Eve_loss: 7.437191
14900   Alice_bob_loss:  0.0041259695     Eve_loss: 7.489406
15000   Alice_bob_loss:  0.0032786985     Eve_loss: 7.5473795
15100   Alice_bob_loss:  0.010252565     Eve_loss: 7.191091
15200   Alice_bob_loss:  0.0060922597     Eve_loss: 7.3770866
15300   Alice_bob_loss:  0.00

27200   Alice_bob_loss:  0.0044554113     Eve_loss: 7.475475
27300   Alice_bob_loss:  0.0051118066     Eve_loss: 7.454295
27400   Alice_bob_loss:  0.004377752     Eve_loss: 7.47
27500   Alice_bob_loss:  0.004965993     Eve_loss: 7.4582872
27600   Alice_bob_loss:  0.006013309     Eve_loss: 7.381578
27700   Alice_bob_loss:  0.0046694754     Eve_loss: 7.453931
27800   Alice_bob_loss:  0.004960051     Eve_loss: 7.4387875
27900   Alice_bob_loss:  0.0050420174     Eve_loss: 7.461675
28000   Alice_bob_loss:  0.0048376373     Eve_loss: 7.461602
28100   Alice_bob_loss:  0.004748526     Eve_loss: 7.4523478
28200   Alice_bob_loss:  0.0054259016     Eve_loss: 7.445877
28300   Alice_bob_loss:  0.004787991     Eve_loss: 7.443311
28400   Alice_bob_loss:  0.0055480925     Eve_loss: 7.458938
28500   Alice_bob_loss:  0.0051146075     Eve_loss: 7.440533
28600   Alice_bob_loss:  0.0050793886     Eve_loss: 7.43086
28700   Alice_bob_loss:  0.005681618     Eve_loss: 7.4027367
28800   Alice_bob_loss:  0.00498

40700   Alice_bob_loss:  0.0047346354     Eve_loss: 7.4541097
40800   Alice_bob_loss:  0.0047703036     Eve_loss: 7.4493103
40900   Alice_bob_loss:  0.00534089     Eve_loss: 7.4267664
41000   Alice_bob_loss:  0.006077546     Eve_loss: 7.4199038
41100   Alice_bob_loss:  0.005422768     Eve_loss: 7.410333
41200   Alice_bob_loss:  0.005261914     Eve_loss: 7.4351206
41300   Alice_bob_loss:  0.009040753     Eve_loss: 7.2384377
41400   Alice_bob_loss:  0.0056938296     Eve_loss: 7.3967476
41500   Alice_bob_loss:  0.00450538     Eve_loss: 7.4643316
41600   Alice_bob_loss:  0.0055804835     Eve_loss: 7.4029284
41700   Alice_bob_loss:  0.0056205965     Eve_loss: 7.4179363
41800   Alice_bob_loss:  0.005624818     Eve_loss: 7.432817
41900   Alice_bob_loss:  0.0055783344     Eve_loss: 7.430791
42000   Alice_bob_loss:  0.005507999     Eve_loss: 7.4183393
42100   Alice_bob_loss:  0.0053579183     Eve_loss: 7.4137278
42200   Alice_bob_loss:  0.00551927     Eve_loss: 7.4222283
42300   Alice_bob_loss:

54100   Alice_bob_loss:  0.0030885485     Eve_loss: 7.5584116
54200   Alice_bob_loss:  0.0034145797     Eve_loss: 7.5396357
54300   Alice_bob_loss:  0.0035000802     Eve_loss: 7.5637016
54400   Alice_bob_loss:  0.003275801     Eve_loss: 7.54433
54500   Alice_bob_loss:  0.002039103     Eve_loss: 7.642967
54600   Alice_bob_loss:  0.0029337516     Eve_loss: 7.5724354
54700   Alice_bob_loss:  0.0030764414     Eve_loss: 7.570863
54800   Alice_bob_loss:  0.0028225495     Eve_loss: 7.576127
54900   Alice_bob_loss:  0.0028440426     Eve_loss: 7.5723205
55000   Alice_bob_loss:  0.002864454     Eve_loss: 7.5934153
55100   Alice_bob_loss:  0.0029686005     Eve_loss: 7.586692
55200   Alice_bob_loss:  0.00766124     Eve_loss: 7.4715724
55300   Alice_bob_loss:  0.0029100555     Eve_loss: 7.5690203
55400   Alice_bob_loss:  0.0023430306     Eve_loss: 7.615801
55500   Alice_bob_loss:  0.0023349738     Eve_loss: 7.615124
55600   Alice_bob_loss:  0.0031586834     Eve_loss: 7.5578446
55700   Alice_bob_los

67500   Alice_bob_loss:  0.00270014     Eve_loss: 7.587865
67600   Alice_bob_loss:  0.0022335264     Eve_loss: 7.6239843
67700   Alice_bob_loss:  0.002738588     Eve_loss: 7.585483
67800   Alice_bob_loss:  0.0037129065     Eve_loss: 7.5572605
67900   Alice_bob_loss:  0.002909869     Eve_loss: 7.5785027
68000   Alice_bob_loss:  0.0029210781     Eve_loss: 7.5713663
68100   Alice_bob_loss:  0.0029025439     Eve_loss: 7.5759983
68200   Alice_bob_loss:  0.0025254278     Eve_loss: 7.599039
68300   Alice_bob_loss:  0.0032051036     Eve_loss: 7.5692296
68400   Alice_bob_loss:  0.002536593     Eve_loss: 7.5988607
68500   Alice_bob_loss:  0.0027747594     Eve_loss: 7.58249
68600   Alice_bob_loss:  0.0026535613     Eve_loss: 7.5888176
68700   Alice_bob_loss:  0.0027884082     Eve_loss: 7.5792036
68800   Alice_bob_loss:  0.0031039657     Eve_loss: 7.561712
68900   Alice_bob_loss:  0.002912688     Eve_loss: 7.57146
69000   Alice_bob_loss:  0.003034979     Eve_loss: 7.559337
69100   Alice_bob_loss: 